In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets
from lib import calculate_normalize as CalNorm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

C:\Users\Owner\AppData\Local\Temp\ipykernel_16908\821615896.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
EPOCHS = 50
BATCH_SIZE = 64

In [5]:
# cuda 확인

In [6]:
trainset = datasets.CIFAR100('./.data',
                             train=True,
                             download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor()
                             ]))
testset = datasets.CIFAR100('./.data',
                             train=False,
                             download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor()
                             ]))
mean_, std_ = CalNorm(trainset)
print("mean = {}\nstd = {}".format(mean_, std_))

Files already downloaded and verified
Files already downloaded and verified
mean = (0.50707513, 0.48654887, 0.44091785)
std = (0.20079845, 0.19834627, 0.20219834)


In [7]:
trainset = datasets.CIFAR100('./.data',
                             train=True,
                             download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean_, std_)
                             ]))
testset = datasets.CIFAR100('./.data',
                             train=False,
                             download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean_, std_)
                             ]))

Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(3200, 625)
        self.fc2 = nn.Linear(625, 100)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 3200)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x